In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns     
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [ ]:
#Upload table from SQL, connect to database

conn_str = (username, password)

try:
    # Create the SQLAlchemy engine
    engine = create_engine(conn_str)

    # Define your SQL query to select data from a table
    table_name = 'View' 
    query = f"SELECT * FROM {table_name}"
    
    # Load data into a DataFrame
    df_original= pd.read_sql(query, engine)
    
    # Close the connection
    #conn.close()
except Exception as ex:
    print("Error:", ex)

In [ ]:
df= df_original.copy()

# Exploratory Data Analysis

In [ ]:
#check top rows, shape, and info of table
df.head()
df.shape
df.info()

In [ ]:
#convert columns to integer type
columns_to_convert = ['Column4','Column5'] 
df[columns_to_convert] = df[columns_to_convert].astype(int)

In [ ]:
df.describe().T

In [ ]:
#check for missing values
df.isna().sum()

# Mapping and preparing data

In [ ]:
#mapping column from Y/N to 1/0
df['Column2'] = df['Column2'].map({'Y': 1, 'N': 0})

In [ ]:
df = pd.get_dummies(df, columns=["Column10"], drop_first=True)

e_columns = df.columns[df.columns.str.startswith("E_")]
df[e_columns] = df[e_columns].astype(int)

In [ ]:
#Creating a new column: AvgCredits by combining credits/4
df['TotalCredits'] = df['Column6'] + df['Column7'] + df['Column8']+ df_map['Column9']
df['AvgCredit'] = df['TotalCredits'] / 4

In [ ]:
conditions = [
    (df['Column1'].isnull()) & (df["AvgCredit"] == 0),
    (df['Column1'].isnull()) & (df["AvgCredit"] > 0) & (df["AvgCredit"] <= 0.3),
    (df['Column1'].isnull()) & (df["AvgCredit"] > 0.3) & (df["AvgCredit"] <= 0.6),
    (df['Column1'].isnull()) & (df["AvgCredit"] > 0.6) & (df["AvgCredit"] <= 0.9),
    (df['Column1'].isnull()) & (df["AvgCredit"] > 0.9)
]

g_values = [0, 1.0, 2.0, 3.0, 4.0]

df['Column1'] = np.select(conditions, g_values, default=df['Column1'])

In [ ]:
# using columns to predict graduation... columns renamed for sensitivity
#best columns were chosen from previously checking correlation, and variance factors

X = df[['Column1','Column2','Column3','Column4','Column5','Column6','Column7','Column8','Column9','Column10']]

In [ ]:
#scaling columns
X.loc[:, ['Column1', 'Column3']] = scaler.transform(X[['Column1', 'Column3']])

* Model chosen from previous training then saved

In [ ]:
# balancing weights, best model chosen through previous training of the models
#model used on previous data and then saved to use to predict

#weights = {
#    0: 5.0,  
#    1: 4.0,  
#    2: 1.0  
#}

#dt_model = DecisionTreeClassifier(
#    max_depth=5,
#    class_weight= weights,          
#    random_state=42)

#dt_model.fit(X_train, y_train)

In [ ]:
#Use model to predict
GraduationPredictions = dt_model.predict(X)

In [ ]:
#Output predictions
print(GraduationPredictions)